# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

*The school district contracted a software engineer to create a model to predict if a student will graduate or not, so it could include this student in an early intervention program and increase the likelihood of her/his graduation. This basically means the software engineer should create a model to predict if the student will pass or fail the exams, and as such, the task is a classification problem, assigning students to either success or fail categories.*

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = None
# IMPLEMENTATION
n_students = student_data.passed.count()

# TODO: Calculate number of features
n_features = None
# IMPLEMENTATION
n_features = len(student_data.columns)

# TODO: Calculate passing students
n_passed = None
# IMPLEMENTATION
n_passed = student_data.passed[student_data.passed == 'yes'].count()

# TODO: Calculate failing students
n_failed = None
# IMPLEMENTATION
n_failed = student_data.passed[student_data.passed == 'no'].count()

# TODO: Calculate graduation rate
grad_rate = None
# IMPLEMENTATION
grad_rate = float(n_passed) / n_students * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here

# TODO: Set the number of training points
num_train = None
# IMPLEMENTATION
num_train = 0.80 * n_students

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train = None
X_test = None
y_train = None
y_test = None
# IMPLEMENTATION
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, 
                                                    y_all,
                                                    stratify = y_all,
                                                    shuffle = True,
                                                    test_size = 0.20, 
                                                    random_state = 13)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 316 samples.
Testing set has 79 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

*According to the page [Choosing the right estimator](http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) at [scikit-learn.org](scikit-learn.org), the suggested estimators are [Linear SVC](http://scikit-learn.org/stable/modules/svm.html#classification), [KNeighbors classifier](http://scikit-learn.org/stable/modules/neighbors.html) and an ensemble method, which will be [Gradient Boosting](http://scikit-learn.org/stable/modules/ensemble.html#gradient-boosting). All three are appropriate to the size of the data set and due to their particular nature, offer different approaches to the problem.*

#### LinearSVC

* *Linear SVC is a support vector machine classifier with a linear kernel, which is generally used in the industry to classify text, hypertext and images. Support vector machine is a kind of machine learning algorithm that tries to classify data by finding the frontier (also known as hyperplane) that separates the data - in our case, those who fail in the exams. [2]*

* *Support Vector Machines are known to be highly effective in high dimensional spaces, meaning that it can handle a great number of features and it is also memory efficient. [3]*

* *This kind of model will likely over-fit when there are many more features than samples in the dataset, therefore some strategies must be in place to avoid over-fitting, like regularization and choosing/tuning properly the kernel function. [3]*

* *This kind of model is a good candidate because our data set has lots of features and a lot more samples, but not enough to use stochastic gradient descent. Support Vector Machines can perform really well in small datasets.*

#### KNeighborsClassifier

* *KNeighborsClassifiers are used in handwritten digits and image classification. This type of classification works by consulting the dataset and assigning a label to the datapoint based on nearest neighbors, hence called instance-based learning or non-generalizing learning. [4]*

* *Its strength comes from the fact that it's one of the simplest, yet powerful machine learning algorithm. As the amount of data increases to infinity, it's guaranteed that a two-class KNeighborsClassifier won't yeld an error rate worse than twice the Bayes error rate. [5]*

* *KNeighborsClassifier may suffer the Curse of Dimensionality, which in KNeighbors context means that calculating the distance of the neighbors to the datapoint may be time consuming and unhelpful when there are lots of features (neighbors) in the dataset. It's also very sensitive to noise, if present in the dataset. [5]*

* *KNeighborsClassifier may be a good candidate, given that students who fail may share similar characteristics and therefore, KNeighborClassifier can compare a student's features to its neighbors and classify him/her as someone who is going to pass or fail at the exams.*

#### Gradient Boosting

* *Gradient Boosting has been used to build ranking models for information retrieval systems, seach engines and recommender systems, to name a few of them. [6],[7]*

* *As a ensemble method, one of its stregnths is using simpler, weaker learners (decision trees usually) and combine them to form strong learners iteratively and at each iteration, putting more weight on where it performed poorly.*

* *Gradient Boosting is known to be harder to fine tune than other methods, but once well adjusted, GB gives good results, usually.*

* *Our dataset has lots of numerical and categorical features, and as such, decision tree based algorithms like Gradient Boosting usually handle them more easily. [8]*

[2]: https://en.wikipedia.org/wiki/Support_vector_machine "Support Vector Machine"
[3]: http://scikit-learn.org/stable/modules/svm.html#classification "SVM Classification"
[4]: http://scikit-learn.org/stable/modules/neighbors.html "Nearest Neighbors"
[5]: https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm "K-nearest neighbors algorithm"
[6]: https://en.wikipedia.org/wiki/Gradient_boosting "Gradient Boosting"
[7]: https://en.wikipedia.org/wiki/Learning_to_rank "Learning to rank"
[8]: https://en.wikipedia.org/wiki/Supervised_learning#Other_factors_to_consider_.28important.29 "Supervised Learning"

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)


def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [21]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from sklearn import model_C
# IMPLEMENTATION
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

# TODO: Initialize the three models
clf_A = None
clf_B = None
clf_C = None
# IMPLEMENTATION
clf_A = LinearSVC(random_state = 13)
clf_B = KNeighborsClassifier()
clf_C = GradientBoostingClassifier(random_state = 13)

# TODO: Set up the training set sizes
X_train_100 = None
y_train_100 = None

X_train_200 = None
y_train_200 = None

X_train_300 = None
y_train_300 = None
# IMPLEMENTATION
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
# IMPLEMENTATION
#train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
#train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
#train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
#train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
#train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
#train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a GradientBoostingClassifier using a training set size of 100. . .
Trained model in 0.1135 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.8288.
Training a GradientBoostingClassifier using a training set size of 200. . .
Trained model in 0.1622 seconds
Made predictions in 0.0015 seconds.
F1 score for training set: 0.9922.
Made predictions in 0.0007 seconds.
F1 score for test set: 0.8596.
Training a GradientBoostingClassifier using a training set size of 300. . .
Trained model in 0.1584 seconds
Made predictions in 0.0016 seconds.
F1 score for training set: 0.9782.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.7788.


### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - LinearSVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :-----------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0166        | 0.0008                 | 0.9265           | 0.8000          |
| 200               | 0.0417        | 0.0005                 | 0.8655           | 0.8000          |
| 300               | 0.0500        | 0.0010                 | 0.7335           | 0.6437          |

** Classifer 2 - KNeighborsClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :-----------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0012        | 0.0020                 | 0.8000           | 0.8475          |
| 200               | 0.0013        | 0.0026                 | 0.8132           | 0.7899          |
| 300               | 0.0016        | 0.0028                 | 0.8643           | 0.8500          |

** Classifer 3 - GradientBoostingClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :-----------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.1677        | 0.0007                 | 1.0000           | 0.8288          |
| 200               | 0.2182        | 0.0008                 | 0.9922           | 0.8596          |
| 300               | 0.1571        | 0.0011                 | 0.9782           | 0.7788          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

*The recommended model based on the experiments is KNeighborsClassifier. The experiment evaluated the performance of candidate models against different sizes of the dataset, and as the dataset size increased, KNeighborsClassifier was the only model that had kept its score improving.*

*Additionally, considering a sceneario of limited resources, KNeighborsClassifier is one of fastest model to train while delivering a quite good performance.*



### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

*KNeighborsClassifier is a type of classification called instance-based learning, which means it stores instances (student information) of the original dataset. This particular process is called training and this type of classification algorithm performs the training process really fast because it will simply store the information.*

*When it's time to predict, or query, if a student will pass or fail, the model uses the stored instances to compute the classification of the queried student. The success or failure of the queried student is then based on the closest and most representative students (neighbors) to the student in question. In layman's terms, it is the mathematical approach to 'a man is known by the company he keeps'.*


### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [9]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
# IMPLEMENTATION:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

# TODO: Create the parameters list you wish to tune
parameters = None
# IMPLEMENTATION
parameters = {'n_neighbors': [5, 10, 15, 30, 45],
              'weights': ['uniform', 'distance'],
              'algorithm': ['ball_tree', 'kd_tree', 'brute'],
              'leaf_size': [5, 10, 15, 30, 45],
              'p': [1, 2]}

# TODO: Initialize the classifier
clf = None
# IMPLEMENTATION
clf = KNeighborsClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = None
# IMPLEMENATION
f1_scorer = make_scorer(f1_score, pos_label = "yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = None
# IMPLEMENTATION:
grid_obj = GridSearchCV(clf, parameters, scoring = f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
#grid_obj = None
# IMPLEMENTATION
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
print "Best parameters: {}".format(grid_obj.best_params_)

Made predictions in 0.1536 seconds.
Tuned model has a training F1 score of 0.8187.
Made predictions in 0.1626 seconds.
Tuned model has a testing F1 score of 0.8154.
Best parameters: {'n_neighbors': 30, 'weights': 'uniform', 'leaf_size': 5, 'algorithm': 'ball_tree', 'p': 1}


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

*The final model's F1 scores are 0.8187 and 0.8154, training and testing respectively. The tuned model's F1 scores are slightly lower than the untuned one but still the best F1 score when compared to the others' F1 scores, which where computed for models trained with 300 data points.*

*This variation is probably due to the added data points when fitting the model, which provides new points to the neighborhood of the student whose success or failure is being predicted.*


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.